In [2]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import mpld3
pd.set_option('display.width', 250)
print('imported necessary libraries')
print('')
df=pd.read_csv('D:\\Pen drive stuff\\DDW-HH06-2900-2011.csv')
print('imported csv file to dataframe')
print('')
print('first 5 rows for inspection')
print('')
print(df.head())
print('')
df['area name']=df['area name'].str.lower()
df['age of the head of household in years']=df['age of the head of household in years'].str.lower()
df['total/rural/urban']=df['total/rural/urban'].str.lower()
print('converted columns \'area name\' , \'age of the head of the household in years\' and \'total/rural/urban\' to lower case')
print('')
print('first 5 rows for inspection')
print('')
print(df.head())
print('')
print('performing data checks...')
print('')
data_check_flag=0
if(df.count().min() == df.shape[0]):
    print('check #1. no missing values found')
else:
    print('check #1. missing values present')
    data_check_flag=1
group_ser=df.groupby('district code')['area name'].nunique()
if(len(group_ser[group_ser>1])>0):
    print('check #2. district code and area name are not one-one, please check data')
    data_check_flag=1
else:
    print('check #2. district code and area name are one-one as expected')
tru_list_df=list(df['total/rural/urban'].unique())
tru_list_ref=['total','rural','urban']
if(len(set(tru_list_ref+tru_list_df))==len(tru_list_ref)):
    print('check #3. values in \'total/rural/urban\' column are as expected')
else:
    print('check #3. unexpected values in \'total/rural/urban\' column')
    data_check_flag=1
print('')
print('the data seems to be a cube extract of sorts and has various levels of aggregation. For instance, the highest level is \'state - karnataka\' \'total\' \'all ages\'')
print('under \'state - karnataka\' and \'total\' \'all ages\', we have lower levels split by various age groups like \'less than 20\',\'20-29\' etc..')
num_of_dist_under_kar=(len(df.loc[df['area name']!='state - karnataka','area name'].unique()))
print('the same pattern repeats for all the',num_of_dist_under_kar,'districts under \'state - karnataka\'')
print('there are 3 distinct values for \'total/rural/urban\'')
print('there are',len(df['age of the head of household in years'].unique()),'different age groups as below :')
print(df['age of the head of household in years'].unique())
print('')
levels_per_district=len(tru_list_ref)*len(df['age of the head of household in years'].unique())
print('therefore there should be ',levels_per_district,'rows per district, let us confirm this')
grouped_df=df.groupby('area name')['area name'].count()
print(grouped_df)
print('')
if (len(grouped_df[grouped_df!=30])==0):
    print('check #4. each area has all expected levels')
else:
    print('check #4. areas have missing levels')
print('')
print('the columns represent the split of households in a district, firstly the split of total households lead by a male vs female member.')
print('the remaining columns show how male and female lead households are split by the various marital statuses as below:')
print(list(df.columns)[6:16])
print('let us verify if the total number of female lead households match the sum of the split by marital status. we can then repeat this for male lead households')
print('')
female_head_col_mismatch=len(df.loc[df['total households - female head']!=df['never married - female head']+df['currently married - female head']+df['widowed - female head']+df['divorced - female head']+df['separated - female head'],:])
male_head_col_mismatch=len(df.loc[df['total households - male head']!=df['never married - male head']+df['currently married - male head']+df['widowed - male head']+df['divorced - male head']+df['separated - male head'],:])
if(female_head_col_mismatch+male_head_col_mismatch==0):
    print('check #5. total male/female headed households match the split by marital status')
else:
    print('check #5. total male/female headed households do not match the split by marital status')
    data_check_flag=1
if(len(df.loc[df['total households - male head']+df['total households - female head']!=df['total households'],:])==0):
    print('check #6. \'total households\' column matches sum of male headed and female headed households')
else:
    print('check #6. \'total households\' column does not match sum of \'total households - female head\' and \'total households- male head\'')
age_total=df.loc[df['age of the head of household in years']=='all ages',['area name','total households']].groupby('area name')['total households'].sum()
age_split=df.loc[df['age of the head of household in years']!='all ages',['area name','total households']].groupby('area name')['total households'].sum()
comp=age_split==age_total
if(len(comp[comp==False])==0):
    print('check #7. row-wise checks- \'all ages\' row\'s sum matches sum of age splits')
else:
    print('check #7. row-wise checks- \'all ages\' row\'s sum does not match sum of age splits')
    data_check_flag=1
total_total=df.loc[df['total/rural/urban']=='total',['area name','total households']].groupby('area name')['total households'].sum()
rural_total=df.loc[df['total/rural/urban']=='rural',['area name','total households']].groupby('area name')['total households'].sum()
urban_total=df.loc[df['total/rural/urban']=='urban',['area name','total households']].groupby('area name')['total households'].sum()
comp=rural_total+urban_total==total_total
if(len(comp[comp==False])==0):
    print('check #8. row-wise checks - sum of rural and urban households match the total households')
else:
    print('check #8. row-wise checks - sum of rural and urban households does not match the total households')
    data_check_flag=1
print('')
if(data_check_flag==1):
    print('one or more data checks failed.')
else:
    print('...all data checks complete')
heat_map_df=df.copy()
heat_map_df=heat_map_df.loc[(heat_map_df['total/rural/urban']=='total')&(heat_map_df['age of the head of household in years']=='all ages')&(heat_map_df['district code']!=0),['area name','total households - male head','total households - female head','total households']]
heat_map_df['per male head']=(heat_map_df['total households - male head']/heat_map_df['total households'])*100
heat_map_df['per female head']=100-heat_map_df['per male head']
heat_map_df=heat_map_df.sort_values('per female head')
heat_map_df['per male head round']=heat_map_df[['per male head']].round(decimals=0).astype(int)
heat_map_df['per female head round']=100-heat_map_df['per male head round']
print('')
print('The heatmap below has the districts sorted in descending order of percentage male-headed households. It is easy to note that bangalore (88%) and bidar(86%) have the highest proportion of male-headed households while udupi(62%) and dakshina kannada(75%) have the lowest proportion')
print('the heatmap also makes it easy to see how little the proportion of male vs female headed households varies for the rest of the areas - betweeen 79% to 85%')
print('')
z=[heat_map_df['per female head round'].values.tolist(),heat_map_df['per male head round'].values.tolist()]
x = heat_map_df['area name'].values.tolist()
y=['%Female H.H','%Male H.H']
z_text=[heat_map_df['per female head round'].values.tolist(),heat_map_df['per male head round'].values.tolist()]
fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text)#, colorscale='Viridis')
py.iplot(fig, filename='annotated_heatmap_text')



imported necessary libraries

imported csv file to dataframe

first 5 rows for inspection

   district code          area name total/rural/urban age of the head of household in years  total households - male head  total households - female head  never married - male head  never married - female head  currently married - male head  \
0              0  State - KARNATAKA             Total                              All Ages                      11053600                         2268615                     293270                        80605                       10419852   
1              0  State - KARNATAKA             Total                          Less than 20                         58886                           28037                      37914                        22395                          20415   
2              0  State - KARNATAKA             Total                                 20-29                        832741                           84337                     165

In [3]:
plt.rcParams.update({'figure.max_open_warning': 0})
scatter_df=df.copy()
scatter_df=scatter_df.loc[(scatter_df['age of the head of household in years']!='all ages')&(scatter_df['age of the head of household in years']!='age not stated')&(scatter_df['district code']!=0)&(scatter_df['total/rural/urban']!='total'),['area name','age of the head of household in years','total/rural/urban','total households - male head','total households - female head']]
scatter_df['color']='b'
scatter_df.loc[scatter_df['total/rural/urban']=='rural','color']='g'
scatter_df['tooltip']=scatter_df['area name']+' , '+scatter_df['age of the head of household in years']+' , '+scatter_df['total/rural/urban']+' , Male head HH:'+scatter_df['total households - male head'].astype(str)+' , Female head HH:'+scatter_df['total households - female head'].astype(str)
scatter_df['size']=0
scatter_df.loc[scatter_df['age of the head of household in years']=='less than 20','size']=50
scatter_df.loc[scatter_df['age of the head of household in years']=='20-29','size']=100
scatter_df.loc[scatter_df['age of the head of household in years']=='30-39','size']=150
scatter_df.loc[scatter_df['age of the head of household in years']=='40-49','size']=200
scatter_df.loc[scatter_df['age of the head of household in years']=='50-59','size']=250
scatter_df.loc[scatter_df['age of the head of household in years']=='60-69','size']=300
scatter_df.loc[scatter_df['age of the head of household in years']=='70-79','size']=350
scatter_df.loc[scatter_df['age of the head of household in years']=='80+','size']=400
tooltip_list=scatter_df['tooltip'].values.tolist()
fig, ax = plt.subplots(subplot_kw=dict(facecolor='#EEEEEE'),figsize=(16,12), dpi=80)
scatter = ax.scatter(scatter_df['total households - male head'].values.tolist(),
                      scatter_df['total households - female head'].values.tolist(),
                      c=scatter_df['color'].values.tolist(),
                     alpha=0.3)
ax.grid(color='white', linestyle='solid')
ax.set_title("female headed vs male headed households (rural and urban) by age groups", size=20)
labels = ['{0}'.format(i) for i in tooltip_list]
tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels=labels)
plt.plot( [0,scatter_df['total households - female head'].max()*1.2],[0,scatter_df['total households - female head'].max()*1.2] )
plt.xlabel('total households - male head')
plt.ylabel('total households - female head')
plt.xticks(np.arange(0, scatter_df['total households - male head'].max()*1.5,100000))
plt.yticks(np.arange(0, scatter_df['total households - female head'].max()+1,10000))
mpld3.plugins.connect(fig, tooltip)
mpld3.show()

In [4]:
scatter_df=scatter_df.loc[scatter_df['area name']!='bangalore',:]
tooltip_list=scatter_df['tooltip'].values.tolist()
fig, ax = plt.subplots(subplot_kw=dict(facecolor='#EEEEEE'),figsize=(16, 12), dpi=80)
scatter = ax.scatter(scatter_df['total households - male head'].values.tolist(),
                     scatter_df['total households - female head'].values.tolist(),
                     c=scatter_df['color'].values.tolist(),
                     #s=10*scatter_df['size'].values.tolist(),
                     alpha=0.3)
ax.grid(color='white', linestyle='solid')
ax.set_title("female headed vs male headed households (rural and urban) by age groups", size=20)
labels = ['{0}'.format(i) for i in tooltip_list]
tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels=labels)
plt.plot( [0,scatter_df['total households - female head'].max()*1.2],[0,scatter_df['total households - female head'].max()*1.2] )
plt.xlabel('total households - male head')
plt.ylabel('total households - female head')
plt.xticks(np.arange(0, scatter_df['total households - male head'].max()*1.3,100000))
plt.yticks(np.arange(0, scatter_df['total households - female head'].max()+1,10000))
mpld3.plugins.connect(fig, tooltip)
mpld3.show()
